In [ ]:
import pandas as pd
import re
import os
from glob import glob
from io import StringIO
from matplotlib import pyplot as plt
import datetime
import requests
import shutil
LOCAL_TIMEZONE = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo

In [ ]:
DROPBOX_URL = "https://www.dropbox.com/sh/f0j8xol7y0qbeg7/AADTapK4KX_nH-0WyUX67rsJa?dl=1"

response = requests.get(DROPBOX_URL)
open("data.zip", "wb").write(response.content)

shutil.unpack_archive('data.zip', 'data')
os.remove('data.zip')

list_of_files = glob("data/*.csv")

In [ ]:
def read_file(filename):
    text = open(filename).read()
    text_splitted = re.split('(.+\n[\*,]+\n)', text)
    text_splitted = text_splitted[1:]
    data = pd.read_csv(StringIO(text_splitted[9]), header=None, parse_dates=[[0, 1]], dayfirst=False)
    data.columns = ['date', 'temperature', 'humidity']
    data = data.set_index('date')
    data = data.tz_localize('UTC')
    data = data.tz_convert(LOCAL_TIMEZONE)
    return data

def remove_begin_end(df, start_delay=pd.Timedelta('20m'), stop_delay=pd.Timedelta('1m')):
    return df[(df.index > df.index.min() + start_delay) & (df.index < df.index.max() - stop_delay)]

In [ ]:
data = [remove_begin_end(read_file(filename)) for filename in list_of_files]
data = pd.concat(data)
data.index.min(), data.index.max()

In [ ]:
data_raw = [read_file(filename) for filename in list_of_files]
data_raw = pd.concat(data_raw)

In [ ]:
len(data), len(data_raw)

In [ ]:
#notes = pd.read_csv('data/notes.txt', header=None, sep=' ', parse_dates=[[0, 1]])
#notes = notes.set_index("0_1")
#notes

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

def myexp(t, a, b, c):
    return a * np.exp(-t / b) + c

def mylin(t, a, b):
    return a + b * t


def fit_span(series, function, p0):
    start = series.index.min()
    x = series.index - start
    x = x.seconds
    y = series.values
    yinf = series.iloc[-1]
    
    popt, pcov = curve_fit(function, x, y, p0=p0)
    
    return popt, pcov

In [ ]:
# riscaldamento 0.25 K/h - 0.44 K/h
# exp b = 1.44625327e+04, 1.04721363e+04

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(template='plotly_white')
#fig.add_trace(
#    go.Scatter(x=data.index, y=data.temperature, name="temperature"),
#    secondary_y=False,
#)

fig.add_trace(
    go.Scatter(x=data_raw.index, y=data_raw.temperature, name="temperature r"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=data.index, y=data.humidity, name="humidity"),
    secondary_y=True,
)

fig.update_xaxes(title_text="date")

# Set y-axes titles
fig.update_yaxes(title_text="temperature", secondary_y=False)
fig.update_yaxes(title_text="humidity", secondary_y=True)



fit_timeranges = [
#    (datetime.datetime(2022, 12, 6, 15, 4), datetime.datetime(2022, 12, 6, 22, 0), myexp),
#    (datetime.datetime(2022, 12, 7, 8, 17), datetime.datetime(2022, 12, 7, 16, 57), myexp), 
#    (datetime.datetime(2022, 12, 11, 9, 43), datetime.datetime(2022, 12, 11, 11, 47), mylin),
#    (datetime.datetime(2022, 12, 12, 9, 30), datetime.datetime(2022, 12, 12, 12, 40), mylin)
]
fit_results = []

for start, stop, function in fit_timeranges:
    start = pd.Timestamp(start).tz_localize(LOCAL_TIMEZONE)
    stop = pd.Timestamp(stop).tz_localize(LOCAL_TIMEZONE)
    data_to_fit_mask = (data.index > start) & (data.index < stop)
    data_to_fit = data[data_to_fit_mask]['temperature']

    if function == myexp:
        p0 = (data_to_fit.iloc[0] - data_to_fit.iloc[-1], 15000, data_to_fit.iloc[-1])
    else:
        p0 = (20, 1E-5)
    popt, pcov = fit_span(data_to_fit, function, p0=p0)
    print(popt)

    xspan_seconds = np.linspace(0, (data_to_fit.index.max() - data_to_fit.index.min()).seconds, 100)
    yfit = function(xspan_seconds, *popt)
    xspan_time = data_to_fit.index.min() + pd.TimedeltaIndex(xspan_seconds, 's')
    fig.add_trace(
        go.Scatter(x=xspan_time, y=yfit, showlegend=False, hoverinfo='none'),
    secondary_y=False
)
    
    fit_results.append(popt)

for v in pd.date_range(data.index.min(), data.index.max(), freq='D', normalize=True)[1:]:
    fig.add_vline(v, line_dash='dot')
    
fig.show()

In [ ]:
import chart_studio.plotly as py
import chart_studio
py.plot(fig)

In [ ]:
import requests
api_key = '4a8f358962854f23e88a914e0072fb87'
base_url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = 'Milan'
complete_url = base_url + "appid=" + api_key + "&q=" + city_name
 
# get method of requests module
# return response object
response = requests.get(complete_url)
response.json()

In [ ]:
url = "https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={key}".format(
    lat='45.46', lon='9.189',
    start=data.index.min().timestamp(),
    end=data.index.max().timestamp(),
    key=api_key   
)

response = requests.get(url)
response.json()